In [14]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [3]:
df = pd.read_csv('datasets\Titanic-Dataset.csv')
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [5]:
df = df [['Pclass', 'Sex', 'Age', 'SibSp', 'Fare', 'Embarked', 'Survived']]
df.head()

,Pclass,Sex,Age,SibSp,Fare,Embarked,Survived
0,3,male,22.0,1,7.2500,S,0
1,1,female,38.0,1,71.2833,C,1
2,3,female,26.0,0,7.9250,S,1
3,1,female,35.0,1,53.1000,S,1
4,3,male,35.0,0,8.0500,S,0


In [8]:
df.isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Fare        0
Embarked    0
Survived    0
dtype: int64

In [7]:
df.dropna(inplace= True)

C:\Users\tawfi\AppData\Local\Temp\ipykernel_50500\1124485253.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace= True)


In [9]:
df.dtypes

Pclass        int64
Sex          object
Age         float64
SibSp         int64
Fare        float64
Embarked     object
Survived      int64
dtype: object

In [12]:
df['Sex'] = LabelEncoder().fit_transform(df['Sex'])
df['Embarked'] = LabelEncoder().fit_transform(df['Embarked'])

C:\Users\tawfi\AppData\Local\Temp\ipykernel_50500\3795814451.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sex'] = LabelEncoder().fit_transform(df['Sex'])
C:\Users\tawfi\AppData\Local\Temp\ipykernel_50500\3795814451.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Embarked'] = LabelEncoder().fit_transform(df['Embarked'])


In [15]:
y = df['Survived']
X = df.drop('Survived', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
X_train.sample(1)

,Pclass,Sex,Age,SibSp,Fare,Embarked
608,2,0,22.0,1,41.5792,0


In [19]:
import torch
from torch.utils.data import Dataset, DataLoader

class TitanicDataset(Dataset):
    def __init__(self, x, y):
        self.X = torch.tensor(x.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_ds = TitanicDataset(X_train, y_train)
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)


In [20]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(X_train.shape[1], 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 1),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)
    
model = Model()
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [21]:
#train the model
for epoch in range(100):
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X).squeeze()
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/100], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 0.6129
Epoch [20/100], Loss: 0.7039
Epoch [30/100], Loss: 0.4396
Epoch [40/100], Loss: 0.6293
Epoch [50/100], Loss: 0.4178
Epoch [60/100], Loss: 0.5025
Epoch [70/100], Loss: 0.2744
Epoch [80/100], Loss: 0.5425
Epoch [90/100], Loss: 0.4498
Epoch [100/100], Loss: 0.3260


In [24]:
# save the model
torch.save(model.state_dict(), 'models/titanic_model.pth')
print("Model saved")

Model saved


In [26]:
import torch.onnx

dummy_input = torch.randn(1, X_train.shape[1])

torch.onnx.export(
    model,                          
    dummy_input,                    
    "models/model.onnx",           
    export_params=True,             
    opset_version=11,          
    input_names=['input'],    
    output_names=['output'],   
    dynamic_axes={                 
        'input': {0: 'batch_size'},
        'output': {0: 'batch_size'}
    }
)

print("ONNX model exported successfully")

ONNX model exported successfully
